## 使用输入datasource

- 输入1: m3 使用 DAI SQL
- 输入2: m3 使用 DataSource
- m1 的 表达式中通过 input_1 和 input_2 引用这两个输入
    - 注意，对于 .* 来合并所有列是要去除 date, instrument
- DAI 文档: https://bigquant.com/wiki/doc/dai-PLSbc1SbZX
- 数据字典: https://bigquant.com/data/home

In [1]:
from bigmodule import M

# <aistudiograph>

# @module(position="101,155", comment='', comment_collapsed=True)
m3 = M.input_features_dai.v15(
    mode='表达式',
    expr="""cn_stock_bar1d.close AS f1
cn_stock_valuation.float_market_cap / cn_stock_bar1d.volume AS f2""",
    expr_tables='',
    expr_drop_na=True,
    sql=''
)

# @module(position="424,154", comment='', comment_collapsed=True)
m4 = M.dai_datasource.v3(
    id='cn_stock_status'
)

# @module(position="226,261", comment='', comment_collapsed=True)
m1 = M.input_features_dai.v15(
    input_1=m3.data,
    input_2=m4.data,
    mode='表达式',
    expr="""-- join输出 input_1 的除date, instrument的所有列，这两个列默认会加上
-- 注意列名不要重复，不然会报错
input_1.* EXCLUDE(date, instrument)
input_2.st_status
cn_stock_bar1d.open AS f3""",
    expr_tables='',
    expr_drop_na=True,
    sql=''
)

# @module(position="244,375", comment='', comment_collapsed=True)
m2 = M.extract_data_dai.v13(
    sql=m1.data,
    start_date='2024-01-01',
    start_date_bound_to_trading_date=False,
    end_date='2024-03-31',
    end_date_bound_to_trading_date=False,
    before_start_days=90,
    debug=False
)
# </aistudiograph>

[2024-04-24 12:52:43] [info     ] input_features_dai.v15 开始运行..
[2024-04-24 12:52:44] [info     ] input_features_dai.v15 命中缓存
[2024-04-24 12:52:44] [info     ] input_features_dai.v15 运行完成[1.89s].
[2024-04-24 12:52:45] [info     ] dai_datasource.v3 开始运行..
[2024-04-24 12:52:45] [info     ] dai_datasource.v3 命中缓存
[2024-04-24 12:52:45] [info     ] dai_datasource.v3 运行完成[0.177s].
[2024-04-24 12:52:45] [info     ] input_features_dai.v15 开始运行..
[2024-04-24 12:52:45] [info     ] expr mode
[2024-04-24 12:52:45] [info     ] input_features_dai.v15 运行完成[0.46s].
[2024-04-24 12:52:45] [info     ] extract_data_dai.v13 开始运行..
[2024-04-24 12:52:47] [info     ] data extracted: (295781, 6)
[2024-04-24 12:52:47] [info     ] extract_data_dai.v13 运行完成[2.05s].


In [6]:
print(m1.data.read()["sql"])

CREATE TABLE _t_b4f4609c75ec4ac0a7c5664c6cb2194f AS SELECT
    cn_stock_bar1d.close AS f1,
    cn_stock_valuation.float_market_cap / cn_stock_bar1d.volume AS f2,
    date,
    instrument
FROM cn_stock_bar1d
    JOIN cn_stock_valuation USING(date, instrument)
QUALIFY
    COLUMNS(*) IS NOT NULL;

SELECT
    _t_b4f4609c75ec4ac0a7c5664c6cb2194f.* EXCLUDE(date, instrument),
    cn_stock_status.st_status,
    cn_stock_bar1d.open AS f3,
    date,
    instrument
FROM cn_stock_bar1d
    JOIN _t_b4f4609c75ec4ac0a7c5664c6cb2194f USING(date, instrument)
    JOIN cn_stock_status USING(date, instrument)
QUALIFY
    COLUMNS(*) IS NOT NULL

